In [1]:
import os
import numpy as np
import pandas as pd

filepath = '../data/8/'
filelists = [filepath+a for a in os.listdir(filepath) if a[-1] not in ['s','f']]

from datetime import datetime
X_ppg = []
X_acl = []
y = []
y_participant = []
X_hr_windows = []
import pickle
for i,f in enumerate(filelists):
    data = pickle.load(open(f,'rb'))
    X_ppg.extend([a.values[:,1].reshape(1,-1,1) for a in data['data'].values])
    X_acl.extend([a[:,1:].reshape(1,-1,3) for a in data['acc_window'].values])
    y.extend(list(data['ecg_rr'].values))
    y_participant.extend([i]*data.shape[0])
    data['participant'] = i
    data['timestamp'] = data['start_time'].apply(lambda a:datetime.utcfromtimestamp(a))
    data = data.sort_values('timestamp').reset_index(drop=True)
    hr_windows = [df[['ecg_rr','participant']].values for i,df in 
                  data.groupby(pd.Grouper(key='timestamp',freq='20S')) if df.shape[0]==10]
    X_hr_windows.extend(hr_windows)

X_acl = np.concatenate(X_acl)
X_ppg = np.concatenate(X_ppg)
y = np.array(y)
y_participant = np.array(y_participant)

X_acl.shape,X_ppg.shape,y.shape,y_participant.shape

import pickle
pickle.dump([X_acl,X_ppg,y,y_participant],open('../data/tabular_data.p','wb'))

In [12]:
from scipy import signal
import matplotlib.pyplot as plt

In [ ]:
f, t, Zxx = signal.stft(X_acl, 32, nperseg=64,noverlap=32*1.75,nfft=5000,axis=1)
plt.pcolormesh(t, f[f<4], np.abs(Zxx)[f<4], shading='gouraud')
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.colorbar()
plt.show()

In [ ]:
Zxx

In [17]:
f, t, Zxx = signal.stft(X_ppg[:20], 64, nperseg=128,noverlap=64*1.75,nfft=10000,axis=1)
Zxx.shape

(20, 5001, 1, 33)

In [ ]:
X_acl[0,:,0].reshape(-1).shape

In [37]:
np.array(X_hr_windows)

array([], dtype=float64)

In [39]:
X = np.array(X_hr_windows)[:,:,0]

In [40]:
y = [a[:,1][0] for a in X_hr_windows]

In [41]:
y = np.array(y)

In [42]:
X.shape,y.shape

((4325, 10), (4325,))

In [43]:
import pickle 
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from sklearn.preprocessing import RobustScaler,StandardScaler, MinMaxScaler,QuantileTransformer
import pickle
from scipy.stats import skew,kurtosis,iqr
# from ecg import ecg_feature_computation
import math
# from hrvanalysis import remove_ectopic_beats
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score, \
recall_score,accuracy_score,auc,classification_report,make_scorer,roc_curve
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV, StratifiedKFold
from sklearn import preprocessing,metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from joblib import Parallel,delayed
delta = 0.1
from sklearn.metrics import roc_curve,auc,make_scorer
from copy import deepcopy

In [44]:
def f1Bias_scorer_CV(y_true,y_pred, ret_bias=False):
    probs = y_true
    y = y_pred
    if not ret_bias:
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        return auc(fpr,tpr)
    precision, recall, thresholds = metrics.precision_recall_curve(y, probs)
    
    f1 = 0.0
    for i in range(0, len(thresholds)):
        if not (precision[i] == 0 and recall[i] == 0):
            if np.abs(precision[i]-recall[i])<.1:
                f = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
                if f > f1:
                    f1 = f
                    bias = thresholds[i]

    if ret_bias:
        return f1, bias
    else:
        return f1
def get_results(X,y):
    delta = 0.1
    paramGrid = {
            'svc__min_samples_leaf': [4],
            'svc__max_features': [.7,1],
            'svc__n_estimators': [100,200],
            'svc__criterion':['gini','entropy'],
    }
    clf = Pipeline([('sts',StandardScaler()),('svc', RandomForestClassifier())])
    gkf = StratifiedKFold(n_splits=5)
    grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=list(gkf.split(X,y)),
                               scoring='accuracy',verbose=5)
    grid_search.fit(X,y)
    clf = grid_search.best_estimator_
    
    probs = cross_val_predict(clf,X,y,cv=gkf.split(X,y),n_jobs=20)
    y_pred = np.int64(probs)
    print(classification_report(y,y_pred),confusion_matrix(y,y_pred))
    return classification_report(y,y_pred)

In [45]:
get_results(X,y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  40 | elapsed:    0.9s remaining:   16.2s
[Parallel(n_jobs=-1)]: Done  11 out of  40 | elapsed:    2.6s remaining:    6.8s
[Parallel(n_jobs=-1)]: Done  20 out of  40 | elapsed:    4.4s remaining:    4.4s
[Parallel(n_jobs=-1)]: Done  29 out of  40 | elapsed:    6.2s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   16.0s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   16.6s finished


              precision    recall  f1-score   support

         0.0       0.09      0.05      0.07       261
         1.0       0.30      0.34      0.32       320
         2.0       0.07      0.08      0.08       261
         3.0       0.75      0.59      0.66       324
         4.0       0.05      0.01      0.02       261
         5.0       0.20      0.35      0.26       352
         6.0       0.14      0.07      0.10       295
         7.0       0.14      0.24      0.18       276
         8.0       0.26      0.34      0.29       273
         9.0       0.21      0.21      0.21       311
        10.0       0.11      0.05      0.07       280
        11.0       0.12      0.15      0.13       260
        12.0       0.14      0.07      0.09       272
        13.0       0.14      0.20      0.17       303
        14.0       0.09      0.10      0.09       276

   micro avg       0.20      0.20      0.20      4325
   macro avg       0.19      0.19      0.18      4325
weighted avg       0.20   

'              precision    recall  f1-score   support\n\n         0.0       0.09      0.05      0.07       261\n         1.0       0.30      0.34      0.32       320\n         2.0       0.07      0.08      0.08       261\n         3.0       0.75      0.59      0.66       324\n         4.0       0.05      0.01      0.02       261\n         5.0       0.20      0.35      0.26       352\n         6.0       0.14      0.07      0.10       295\n         7.0       0.14      0.24      0.18       276\n         8.0       0.26      0.34      0.29       273\n         9.0       0.21      0.21      0.21       311\n        10.0       0.11      0.05      0.07       280\n        11.0       0.12      0.15      0.13       260\n        12.0       0.14      0.07      0.09       272\n        13.0       0.14      0.20      0.17       303\n        14.0       0.09      0.10      0.09       276\n\n   micro avg       0.20      0.20      0.20      4325\n   macro avg       0.19      0.19      0.18      4325\nweigh